# AI-Powered Resume Analyzer for Job Postings

This tool is designed to analyze resumes against specific job postings, offering valuable insights such as:

- Identification of skill gaps
- Keyword matching between the CV and the job description
- Tailored recommendations for CV improvement
- An alignment score reflecting how well the CV fits the job
- Personalized feedback 
- Job market trend insights

An example of the tool's output can be found [here](https://tvarol.github.io/sideProjects/AILLMAgents/output.html).

In [ ]:
# Imports
import gradio as gr
import PyPDF2
import io
import os
from dotenv import load_dotenv
from openai import OpenAI

In [ ]:
# Load environment variables
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key
if not api_key:
    print("No API key was found!!!")
else:
    print("API key found and looks good so far!")

In [ ]:
openai = OpenAI()

## Using a Frontier Model GPT-4o Mini for This Project

### Types of Prompts

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [ ]:
# Define our system prompt 
system_prompt = """You are a powerful AI model designed to assist with resume analysis. Your task is to analyze a resume against a given job posting and provide feedback on how well the resume aligns with the job requirements. Your response should include the following: 
1) Skill gap identification: Compare the skills listed in the resume with those required in the job posting, highlighting areas where the resume may be lacking or overemphasized.
2) Keyword matching between a CV and a job posting: Match keywords from the job description with the resume, determining how well they align. Provide specific suggestions for missing keywords to add to the CV.
3) Recommendations for CV improvement: Provide actionable suggestions on how to enhance the resume, such as adding missing skills or rephrasing experience to match job requirements.
4) Alignment score: Display a score that represents the degree of alignment between the resume and the job posting.
5) Personalized feedback: Offer tailored advice based on the job posting, guiding the user on how to optimize their CV for the best chances of success.
6) Job market trend insights, provide broader market trends and insights, such as in-demand skills and salary ranges.
Provide responses that are concise, clear, and to the point. Respond in markdown."""

### Use Gradio for UI to upload CV and paste the text for job posting


In [ ]:
# The job posting and the CV are required to define the user prompt
# The user will input the job posting as text in a box here
# The user will upload the CV in PDF format, from which the text will be extracted

def process_pdf(pdf_file):
    """Extract text from PDF file"""
    if pdf_file is None:
        return None
    try:
        pdf_reader = PyPDF2.PdfReader(io.BytesIO(pdf_file))
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
        return text
    except Exception as e:
        return f"Error processing PDF: {str(e)}"

def analyze_resume(job_posting, pdf_file, progress=gr.Progress()):
    """Analyze resume against job posting with streaming output"""
    
    # Process the PDF
    cv_text = process_pdf(pdf_file)
    if cv_text is None:
        yield "Please upload a CV first."
        return
    if cv_text.startswith("Error"):
        yield cv_text
        return
    
    # Create user prompt
    user_prompt = f"""
    Job Posting:
    {job_posting}

    CV:
    {cv_text}
    """
    
    # Create messages for API
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    # Call OpenAI API with streaming
    try:
        stream = openai.chat.completions.create(
            model="gpt-4o-mini",  
            messages=messages,
            stream=True
        )
        
        # Initialize response text
        response_text = ""
        
        # Stream the response
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                response_text += chunk.choices[0].delta.content
                yield response_text
                
    except Exception as e:
        yield f"Error: {str(e)}"
    

In [ ]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

view = gr.Interface(
    fn=analyze_resume,
   inputs=[
        gr.Textbox(label="Job Posting",
                placeholder="Paste the job posting text here...",
                lines=10),
        gr.File(label="Upload CV (PDF)",
                file_types=[".pdf"],
                type="binary")
    ],
    outputs=[gr.Markdown(label="Analysis Result:")],
    flagging_mode="never",
    js=force_dark_mode
    #live=True
)

view.launch(share=True)

In [ ]:
## If you would like to save the response content as a Markdown file, uncomment the following lines
#with open('yourfile.md', 'w') as file:
#    file.write(response.choices[0].message.content)

## You can then run the line below to create output.html which you can open on your browser
#!pandoc yourfile.md -o output.html